In [13]:
from datasets import load_dataset
import os
import re
from transformers import AutoTokenizer
import random
import torch 
def get_files_in_directory(directory, keyword):
    file_list = []
    for file in os.listdir(directory):
        if re.search(keyword, file):
            file_list.append(os.path.join(directory, file))
    return file_list

traindata_directory = './c4/realnewslike/'

traindata_path = get_files_in_directory(traindata_directory, r'train')
valdata_path = get_files_in_directory(traindata_directory, r'validation')

traindata = load_dataset('json', data_files={'train': traindata_path}, split='train')
valdata = load_dataset('json', data_files={'validation': valdata_path}, split='validation')

Resolving data files:   0%|          | 0/512 [00:00<?, ?it/s]

Found cached dataset json (/home/dujh/.cache/huggingface/datasets/json/default-cd15a4614eb5e5ca/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
Found cached dataset json (/home/dujh/.cache/huggingface/datasets/json/default-f4086d6aa7a60284/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [21]:
def get_c4(nsamples, seed, seqlen, model):
    from datasets import load_dataset
    import os
    import re
    def get_files_in_directory(directory, keyword):
        file_list = []
        for file in os.listdir(directory):
            if re.search(keyword, file):
                file_list.append(os.path.join(directory, file))
        return file_list

    traindata_directory = './c4/realnewslike/'

    traindata_path = get_files_in_directory(traindata_directory, r'train')
    valdata_path = get_files_in_directory(traindata_directory, r'validation')

    traindata = load_dataset('json', data_files={'train': traindata_path}, split='train')
    valdata = load_dataset('json', data_files={'validation': valdata_path}, split='validation')

    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(model, use_fast=False)

    import random
    random.seed(seed)
    print("seed="+str(seed))
    trainloader = []
    for _ in range(nsamples):
        while True:
            i = random.randint(0, len(traindata) - 1)
            trainenc = tokenizer(traindata[i]['text'], return_tensors='pt')
            if trainenc.input_ids.shape[1] >= seqlen:
                break
        # print(trainenc.input_ids.shape[1] - seqlen - 1)
        i = random.randint(0, trainenc.input_ids.shape[1] - seqlen - 1)
        j = i + seqlen
        inp = trainenc.input_ids[:, i:j]
        tar = inp.clone()
        tar[:, :-1] = -100
        trainloader.append((inp, tar))

    import random
    random.seed(0)
    valenc = []
    for _ in range(256):
        while True:
            i = random.randint(0, len(valdata) - 1)
            tmp = tokenizer(valdata[i]['text'], return_tensors='pt')
            if tmp.input_ids.shape[1] >= seqlen:
                break
        i = random.randint(0, tmp.input_ids.shape[1] - seqlen - 1)
        j = i + seqlen
        valenc.append(tmp.input_ids[:, i:j])
    valenc = torch.hstack(valenc)
    class TokenizerWrapper:
        def __init__(self, input_ids):
            self.input_ids = input_ids
    valenc = TokenizerWrapper(valenc)

    return trainloader, valenc 
    # return  valenc 

In [22]:
train,vance =get_c4(128,0,1024,"opt-125m")

Resolving data files:   0%|          | 0/512 [00:00<?, ?it/s]

Found cached dataset json (/home/dujh/.cache/huggingface/datasets/json/default-cd15a4614eb5e5ca/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
Found cached dataset json (/home/dujh/.cache/huggingface/datasets/json/default-f4086d6aa7a60284/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


seed=0


In [9]:
assert type(2) in  [int,float]

In [12]:
def get_wikitext2(nsamples, seed, seqlen, model):
    import ssl
    from datasets import load_dataset
    # ssl._create_default_https_context = ssl._create_unverified_context
    ssl._create_default_https_context = ssl._create_unverified_context
    traindata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
    testdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')

    return traindata, testdata

In [ ]:
model_path="./opt-125m/"
train,test= get_wikitext2(128,0,2048,model_path)

In [15]:
import ssl
from datasets import load_dataset
# ssl._create_default_https_context = ssl._create_unverified_context
ssl._create_default_https_context = ssl._create_unverified_context
# traindata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
# testdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')


In [18]:
train_dataset = load_dataset('imdb', split='train')
# valdata = load_dataset(        'allenai/c4', 'allenai--c4', data_files={'validation': 'en/c4-validation.00000-of-00008.json.gz'}, split='validation')

ConnectionError: Couldn't reach 'imdb' on the Hub (SSLError)